In [5]:
from DataProcessing import encodedShake, code2sonnet
import nltk
from nltk.corpus import cmudict
import random

In [7]:
d = cmudict.dict()

encodedSonnets, encodedSyllaDict, code2word, punc2code = encodedShake()

In [8]:
def Vowel_num(pron):
    syl = ""
    for elem in pron:
        if "1" in elem or "2" in elem: syl += "1"
        elif "0" in elem: syl += "0"
    return syl

def Rhyme(pron):
    rhy = ""
    v = 0
    for i in range(len(pron) - 1, -1, -1):
        if "1" in pron[i] or "2" in pron[i] or "0" in pron[i]: 
            v = i
            break
    for j in range(v, len(pron)):
        rhy += pron[j]
    return rhy

In [9]:
rhyme_dic = {} # dictionary of rhyme
syllable_dic = {} # dictionary of syllable (e.g. 1, 0, 10, ...)

for i in range(len(code2word)):
    single_word = code2word[i]
    single_word_syl = []
    single_word_rhyme = []
    if single_word in d:
        for pron in d[single_word]:
            single_word_syl.append(Vowel_num(pron)) 
            single_word_rhyme.append(Rhyme(pron)) 
        for s in single_word_syl:
            if s not in syllable_dic:
                syllable_dic[s] = []
            if single_word not in syllable_dic[s]:
                syllable_dic[s].append(single_word)
        for r in single_word_rhyme:
            if r not in rhyme_dic:
                rhyme_dic[r] = []
            if single_word not in rhyme_dic[r]:
                rhyme_dic[r].append(single_word)

In [11]:
# Generate the last word of rhyme
rhyme_lastword = []
key_list = []
key_num = 0
while key_num < 7:
    key = random.choice(list(rhyme_dic.keys()))
    if key not in key_list:
        if len(rhyme_dic[key]) >= 2:
            rhyme_lastword.append(random.sample(rhyme_dic[key], 2))
            key_list.append(key)
            key_num = key_num + 1

In [12]:
rhyme_lastword

[['wrought', 'taught'],
 ['kills', 'wills'],
 ['saw', 'draw'],
 ['thrusts', 'thus'],
 ['am', 'lamb'],
 ['confine', 'pine'],
 ['convert', 'desert']]

In [13]:
# Order the last word in 14 line
rhyme_lastword_order = []

rhyme_lastword_order.append(rhyme_lastword[0][0])
rhyme_lastword_order.append(rhyme_lastword[1][0])
rhyme_lastword_order.append(rhyme_lastword[0][1])
rhyme_lastword_order.append(rhyme_lastword[1][1])
rhyme_lastword_order.append(rhyme_lastword[2][0])
rhyme_lastword_order.append(rhyme_lastword[3][0])
rhyme_lastword_order.append(rhyme_lastword[2][1])
rhyme_lastword_order.append(rhyme_lastword[3][1])
rhyme_lastword_order.append(rhyme_lastword[4][0])
rhyme_lastword_order.append(rhyme_lastword[5][0])
rhyme_lastword_order.append(rhyme_lastword[4][1])
rhyme_lastword_order.append(rhyme_lastword[5][1])
rhyme_lastword_order.append(rhyme_lastword[6][0])
rhyme_lastword_order.append(rhyme_lastword[6][1])

In [14]:
rhyme_lastword_order

['wrought',
 'kills',
 'taught',
 'wills',
 'saw',
 'thrusts',
 'draw',
 'thus',
 'am',
 'confine',
 'lamb',
 'pine',
 'convert',
 'desert']

In [17]:
from HMM_fast_addition import HiddenMarkovModel, unsupervised_HMM

In [18]:
# Reverse the encodedSonnets
encodedSonnets_reverse = []

for a in encodedSonnets:
    a.reverse()
    encodedSonnets_reverse.append(a)

In [19]:
# Train the HMM using the reversed encodedSonnets and return A and O

A,O = unsupervised_HMM(encodedSonnets_reverse, 40, 100)

Iteration: #  1; Took 0.93s
Iteration: #  2; Took 0.88s
Iteration: #  3; Took 0.85s
Iteration: #  4; Took 0.86s
Iteration: #  5; Took 0.85s
Iteration: #  6; Took 0.83s
Iteration: #  7; Took 0.87s
Iteration: #  8; Took 0.95s
Iteration: #  9; Took 1.01s
Iteration: # 10; Took 0.99s
Iteration: # 11; Took 0.94s
Iteration: # 12; Took 0.92s
Iteration: # 13; Took 0.91s
Iteration: # 14; Took 0.97s
Iteration: # 15; Took 0.98s
Iteration: # 16; Took 0.98s
Iteration: # 17; Took 0.88s
Iteration: # 18; Took 0.97s
Iteration: # 19; Took 1.01s
Iteration: # 20; Took 0.92s
Iteration: # 21; Took 0.93s
Iteration: # 22; Took 0.91s
Iteration: # 23; Took 0.87s
Iteration: # 24; Took 0.93s
Iteration: # 25; Took 0.99s
Iteration: # 26; Took 0.89s
Iteration: # 27; Took 0.90s
Iteration: # 28; Took 0.95s
Iteration: # 29; Took 0.92s
Iteration: # 30; Took 0.85s
Iteration: # 31; Took 0.86s
Iteration: # 32; Took 0.94s
Iteration: # 33; Took 0.98s
Iteration: # 34; Took 0.89s
Iteration: # 35; Took 0.89s
Iteration: # 36; Too

In [20]:
# Generate the 14 line in the reverse order (the first word of each line is of rhyme),each line of 8 words

import numpy as np

def generate_14line(A,O,rhyme_lastword_order,syllable_dic):
    O = np.asarray(O)
    A = np.asarray(A)
    emission = []
    states = []
    
    for line in range(14):
        syllable_count = 0
        emission_initial = rhyme_lastword_order[line]
        emission_initial_id = [k for k,v in code2word.items() if v == emission_initial][0]
        emission.append(emission_initial_id)
        state_initial_maxlikelihood = np.where(O == np.max(O[:,emission_initial_id]))[0][0]
        states.append(state_initial_maxlikelihood)
        
        state_gen = state_initial_maxlikelihood
        while syllable_count < 7:
            ran = random.uniform(0, 1)
            A_row = A[state_gen]
            for j in range(len(A_row)):
                ran = ran - A_row[j]
                if ran <= 1e-6:
                    break
            state_gen = j
            states.append(state_gen)
            
            ran = random.uniform(0, 1)
            O_row = O[state_gen]
            for j in range(len(O_row)):
                ran = ran - O_row[j]
                if ran <= 1e-6:
                    break
            emission.append(j)
            
            syllable_count = syllable_count + 1
        
    return emission,states

In [21]:
emission,states = generate_14line(A,O,rhyme_lastword_order,syllable_dic)

In [22]:
# Print each line in the reverse order

for line in range(14):
    emission_line = emission[line*8:line*8+8]
    emission_line.reverse()
    print(' '.join(code2sonnet(emission_line, code2word)))

lived kills keen grows take yours faculty wrought
graves anon evil whereto souls importune lovely kills
deserved endeared mind's anew and merit bosom taught
prouder owes act pleasing lesson fulness object wills
merit approve wilfulness gust weed insufficiency kind-hearted saw
conscience reserve gain put'st tables wantonly abide thrusts
kingly beloved injurious feathered dun seek accidents draw
freedom dressing above another's compounded ne'er page thus
reckon conquest slumbers death-bed angry kingdoms owe am
rosy creature child vile hath augurs revolution confine
judgment's terms a turn respects wilfully liberty lamb
broken fickle war's thrivers head faster send pine
abused lovely budding powers verse bitterness why convert
straight power marigold graven tops fast patient desert
